# PS4 → PS5 Migration Experimentation POC (Synthetic)

**Goal:** Demonstrate how to design, analyze, and communicate an experimentation + causal inference readout for a migration campaign.

This notebook uses a **synthetic dataset** generated to resemble a realistic product marketing experiment:
- Stratified randomization
- Conversion + revenue outcomes
- Guardrail metrics
- Variance reduction (CUPED)
- Bayesian analysis
- Sequential testing / early stopping

> Replace the synthetic data with real tables in production. The analysis pattern stays the same.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats

DATA_PATH = "data/migration_experiment_synthetic.csv"
df = pd.read_csv(DATA_PATH)

df.head()


,user_id,country,platform_tenure_days,last_30d_sessions,last_90d_spend,ps_plus_member,pre_30d_spend,spend_tier,engagement_tier,stratum,treatment,ps5_purchase_30d,revenue_60d,post_30d_sessions,unsubscribe_flag,optout_flag
0,1,DE,2691,12,16.53,0,15.66,S1_low,E4_vhigh,S1_low_E4_vhigh,1,0,51.29,9,0,0
1,2,US,3223,8,43.08,1,12.27,S2_mid,E2_mid,S2_mid_E2_mid,1,0,4.86,13,0,0
2,3,FR,2099,16,34.08,1,10.39,S2_mid,E4_vhigh,S2_mid_E4_vhigh,1,1,113.93,22,0,0
3,4,UK,1528,6,76.28,0,27.88,S4_vhigh,E1_low,S4_vhigh_E1_low,1,0,0.00,9,0,0
4,5,US,46,8,98.47,0,27.99,S4_vhigh,E2_mid,S4_vhigh_E2_mid,1,0,56.02,9,0,0


## 1) Quick sanity checks

We confirm:
- Treatment/control split is ~50/50 overall and within strata
- Basic covariates are balanced
- Guardrails are not exploding


In [2]:
# Overall split
df["treatment"].value_counts(normalize=True)



treatment
1    0.50008
0    0.49992
Name: proportion, dtype: float64

In [3]:
# Split by stratum (spot check)
check = (df.groupby(["stratum","treatment"])["user_id"].count()
         .unstack(fill_value=0))
check["treat_rate"] = check[1] / (check[0] + check[1])
check.sort_values("treat_rate").head(10), check["treat_rate"].describe()


(treatment            0     1  treat_rate
 stratum                                 
 S2_mid_E1_low     1951  1951    0.500000
 S2_mid_E2_mid     1749  1749    0.500000
 S2_mid_E3_high    1426  1426    0.500000
 S2_mid_E4_vhigh   1124  1124    0.500000
 S3_high_E1_low    1954  1954    0.500000
 S3_high_E2_mid    1741  1741    0.500000
 S3_high_E4_vhigh  1150  1150    0.500000
 S4_vhigh_E1_low   1947  1947    0.500000
 S1_low_E1_low     1962  1963    0.500127
 S4_vhigh_E2_mid   1771  1772    0.500141,
 count    16.000000
 mean      0.500086
 std       0.000092
 min       0.500000
 25%       0.500000
 50%       0.500064
 75%       0.500178
 max       0.500216
 Name: treat_rate, dtype: float64)

In [4]:
# Covariate balance (standardized mean differences)
def smd(x_t, x_c):
    x_t = np.asarray(x_t); x_c = np.asarray(x_c)
    return (x_t.mean() - x_c.mean()) / np.sqrt(0.5*(x_t.var(ddof=1)+x_c.var(ddof=1)))

covars = ["platform_tenure_days","last_30d_sessions","last_90d_spend","ps_plus_member","pre_30d_spend"]
out = []
t = df[df.treatment==1]
c = df[df.treatment==0]
for col in covars:
    out.append((col, smd(t[col], c[col])))
pd.DataFrame(out, columns=["feature","SMD"]).sort_values("SMD", key=lambda s: s.abs(), ascending=False)


,feature,SMD
4,pre_30d_spend,-1.638387e-02
3,ps_plus_member,1.640815e-03
0,platform_tenure_days,1.207920e-03
2,last_90d_spend,-9.867435e-04
1,last_30d_sessions,9.292302e-07


## 2) Primary metric: 30-day PS5 conversion

We estimate:
- Conversion rates by group
- Absolute and relative lift
- 95% CI
- Frequentist p-value (two-proportion z test)


In [5]:
def conversion_summary(d):
    n = len(d)
    x = d["ps5_purchase_30d"].sum()
    rate = x/n
    return n, x, rate

n_c, x_c, p_c = conversion_summary(df[df.treatment==0])
n_t, x_t, p_t = conversion_summary(df[df.treatment==1])

abs_lift = p_t - p_c
rel_lift = abs_lift / p_c

# Two-proportion z test
p_pool = (x_c + x_t) / (n_c + n_t)
se_pool = np.sqrt(p_pool*(1-p_pool)*(1/n_c + 1/n_t))
z = abs_lift / se_pool
p_value = 2*(1 - stats.norm.cdf(abs(z)))

# Wald CI on difference (ok for large n)
se_diff = np.sqrt(p_c*(1-p_c)/n_c + p_t*(1-p_t)/n_t)
ci = (abs_lift - 1.96*se_diff, abs_lift + 1.96*se_diff)

{
    "control_rate": p_c,
    "treatment_rate": p_t,
    "absolute_lift": abs_lift,
    "relative_lift": rel_lift,
    "z": z,
    "p_value": p_value,
    "95%_CI_abs_lift": ci
}


{'control_rate': np.float64(0.1751480236837894),
 'treatment_rate': np.float64(0.21048632218844984),
 'absolute_lift': np.float64(0.03533829850466044),
 'relative_lift': np.float64(0.20176247360038657),
 'z': np.float64(10.014733579854493),
 'p_value': np.float64(0.0),
 '95%_CI_abs_lift': (np.float64(0.028429199164298577),
  np.float64(0.04224739784502231))}

## 3) Guardrails

We check whether treatment meaningfully increases unsubscribe / opt-out rates.


In [6]:
def rate(col):
    g = df.groupby("treatment")[col].mean()
    return float(g.loc[0]), float(g.loc[1]), float(g.loc[1]-g.loc[0])

for col in ["unsubscribe_flag","optout_flag"]:
    c_rate, t_rate, diff = rate(col)
    print(col, {"control": c_rate, "treatment": t_rate, "diff": diff})


unsubscribe_flag {'control': 0.0043206913106096975, 'treatment': 0.0074388097904335305, 'diff': 0.003118118479823833}
optout_flag {'control': 0.005880940950552089, 'treatment': 0.011238201887697969, 'diff': 0.00535726093714588}


## 4) Variance reduction with CUPED (on revenue)

CUPED uses a **pre-period metric** correlated with the outcome to reduce variance.

Here:
- Outcome = `revenue_60d`
- Covariate = `pre_30d_spend`


In [7]:
y = df["revenue_60d"].values
x = df["pre_30d_spend"].values

theta = np.cov(y, x, ddof=1)[0,1] / np.var(x, ddof=1)
y_cuped = y - theta*(x - x.mean())

df2 = df.copy()
df2["revenue_60d_cuped"] = y_cuped

def diff_in_means(d, col):
    t = d[d.treatment==1][col].values
    c = d[d.treatment==0][col].values
    diff = t.mean() - c.mean()
    # Welch t CI
    se = np.sqrt(t.var(ddof=1)/len(t) + c.var(ddof=1)/len(c))
    ci = (diff - 1.96*se, diff + 1.96*se)
    return diff, ci, se

raw_diff, raw_ci, raw_se = diff_in_means(df2, "revenue_60d")
cuped_diff, cuped_ci, cuped_se = diff_in_means(df2, "revenue_60d_cuped")

{
    "raw_revenue_diff": raw_diff,
    "raw_95%_CI": raw_ci,
    "raw_SE": raw_se,
    "cuped_revenue_diff": cuped_diff,
    "cuped_95%_CI": cuped_ci,
    "cuped_SE": cuped_se,
    "SE_reduction_factor": cuped_se/raw_se
}


{'raw_revenue_diff': np.float64(9.406110424092432),
 'raw_95%_CI': (np.float64(8.458525858204244), np.float64(10.353694989980621)),
 'raw_SE': np.float64(0.48346151320825986),
 'cuped_revenue_diff': np.float64(9.511686330685087),
 'cuped_95%_CI': (np.float64(8.570941792773485),
  np.float64(10.45243086859669)),
 'cuped_SE': np.float64(0.4799717030161237),
 'SE_reduction_factor': np.float64(0.9927816173639599)}

## 5) Bayesian readout (conversion)

Bayesian analysis answers questions like:

- “Given the data, what's the probability treatment is better than control?”
- “What's the probability lift is at least X?”

For conversion we can use a **Beta-Binomial** model:
- Prior: Beta(1, 1) (uniform)
- Posterior: Beta(1 + conversions, 1 + nonconversions)


In [8]:
from scipy.stats import beta

a0, b0 = 1, 1  # prior

post_c = (a0 + x_c, b0 + (n_c - x_c))
post_t = (a0 + x_t, b0 + (n_t - x_t))

# Monte Carlo to estimate distribution of lift
draws = 200000
pc = beta.rvs(post_c[0], post_c[1], size=draws, random_state=42)
pt = beta.rvs(post_t[0], post_t[1], size=draws, random_state=43)
lift = pt - pc

prob_lift_gt0 = float((lift > 0).mean())
prob_lift_gt_001 = float((lift > 0.01).mean())  # > 1pp absolute
cred_int = (float(np.quantile(lift, 0.025)), float(np.quantile(lift, 0.975)))

{
    "P(lift > 0)": prob_lift_gt0,
    "P(lift > 1pp absolute)": prob_lift_gt_001,
    "95%_credible_interval_lift": cred_int
}


{'P(lift > 0)': 1.0,
 'P(lift > 1pp absolute)': 1.0,
 '95%_credible_interval_lift': (0.028422807315936934, 0.042239109446691694)}

## 6) Sequential testing (multiple peeks without lying to yourself)

In practice, teams check results daily. If you stop “as soon as p < 0.05”, false positives explode.

Two common fixes:

### A) Frequentist group sequential testing
Plan a small number of “looks” (e.g., day 7, 14, 21, 30) with stricter thresholds early.
A simple option: **O'Brien-Fleming** boundaries.

### B) Bayesian sequential monitoring
Bayes updates are coherent under continuous monitoring. You choose a decision rule like:
- Stop for success if P(lift > 0) ≥ 0.95
- Stop for futility if P(lift > 0) ≤ 0.10


In [9]:
# We'll simulate interim looks by ordering users randomly (as if they arrived over time)
d = df.sample(frac=1, random_state=7).reset_index(drop=True)

looks = [0.25, 0.50, 0.75, 1.00]  # 4 looks
alpha = 0.05

def obrien_fleming_crit(alpha, t):
    # approximate two-sided OBF boundary using normal quantiles
    # critical z at information fraction t
    z_alpha2 = stats.norm.ppf(1 - alpha/2)
    return z_alpha2 / np.sqrt(t)

results = []
for frac in looks:
    m = int(len(d)*frac)
    dd = d.iloc[:m]
    n_c_i = (dd.treatment==0).sum()
    n_t_i = (dd.treatment==1).sum()
    x_c_i = dd.loc[dd.treatment==0, "ps5_purchase_30d"].sum()
    x_t_i = dd.loc[dd.treatment==1, "ps5_purchase_30d"].sum()

    p_c_i = x_c_i/n_c_i
    p_t_i = x_t_i/n_t_i
    lift_i = p_t_i - p_c_i

    p_pool_i = (x_c_i + x_t_i) / (n_c_i + n_t_i)
    se_pool_i = np.sqrt(p_pool_i*(1-p_pool_i)*(1/n_c_i + 1/n_t_i))
    z_i = lift_i / se_pool_i

    z_crit = obrien_fleming_crit(alpha, frac)
    reject = abs(z_i) >= z_crit

    results.append({
        "look_frac": frac,
        "n": m,
        "lift": lift_i,
        "z": z_i,
        "z_crit_(OBF)": z_crit,
        "reject_at_this_look": reject
    })

pd.DataFrame(results)


,look_frac,n,lift,z,z_crit_(OBF),reject_at_this_look
0,0.25,12500,0.041542,5.918670,3.919928,True
1,0.50,25000,0.039829,7.992380,2.771808,True
2,0.75,37500,0.038692,9.498398,2.263171,True
3,1.00,50000,0.035338,10.014734,1.959964,True


In [10]:
# Bayesian sequential monitoring using Beta-Binomial posteriors
from scipy.stats import beta

def bayes_prob_lift_gt0(dd, draws=60000, seed=123):
    n_c = (dd.treatment==0).sum()
    n_t = (dd.treatment==1).sum()
    x_c = dd.loc[dd.treatment==0, "ps5_purchase_30d"].sum()
    x_t = dd.loc[dd.treatment==1, "ps5_purchase_30d"].sum()
    pc = beta.rvs(1+x_c, 1+(n_c-x_c), size=draws, random_state=seed)
    pt = beta.rvs(1+x_t, 1+(n_t-x_t), size=draws, random_state=seed+1)
    return float((pt-pc > 0).mean()), float((pt-pc > 0.01).mean())

bayes_results = []
for frac in looks:
    m = int(len(d)*frac)
    dd = d.iloc[:m]
    p_gt0, p_gt1pp = bayes_prob_lift_gt0(dd)
    bayes_results.append({
        "look_frac": frac,
        "n": m,
        "P(lift>0)": p_gt0,
        "P(lift>1pp)": p_gt1pp,
        "success_stop_(>=0.95)": p_gt0 >= 0.95,
        "futility_stop_(<=0.10)": p_gt0 <= 0.10
    })

pd.DataFrame(bayes_results)


,look_frac,n,P(lift>0),P(lift>1pp),success_stop_(>=0.95),futility_stop_(<=0.10)
0,0.25,12500,1.0,1.0,True,False
1,0.50,25000,1.0,1.0,True,False
2,0.75,37500,1.0,1.0,True,False
3,1.00,50000,1.0,1.0,True,False


## 7) Heterogeneous effects (who responds?)

A senior-level addition: quantify where the lift comes from.

Here we do a simple segment cut (PS+ membership and spend tier).
In real work you can extend this to uplift models / causal forests.


In [11]:
seg = (df
       .groupby(["ps_plus_member","spend_tier","treatment"])["ps5_purchase_30d"]
       .mean()
       .unstack())
seg["abs_lift"] = seg[1] - seg[0]
seg.sort_values("abs_lift", ascending=False).head(12)


treatment                         0         1  abs_lift
ps_plus_member spend_tier                              
1              S1_low      0.164499  0.226961  0.062461
               S4_vhigh    0.206010  0.262057  0.056047
               S3_high     0.200644  0.251868  0.051224
               S2_mid      0.206101  0.255267  0.049165
0              S2_mid      0.156806  0.186361  0.029555
               S1_low      0.140758  0.161836  0.021078
               S3_high     0.166184  0.182267  0.016083
               S4_vhigh    0.174412  0.185369  0.010957

## Executive Summary

- **Primary result (30-day PS5 conversion):**  
  Control = **17.51%**, Treatment = **21.05%**, Lift = **+3.53 pp** (p < 1e-6, effectively ~0).

- **Bayesian:**  
  **P(lift > 0) = 1.00**, 95% credible interval = **[+2.84 pp, +4.22 pp]**.

- **Revenue impact (60-day):**  
  **+$9.51 per user** (CUPED-adjusted), 95% CI = **[$8.57, $10.45]**.

- **Guardrails:**  
  Unsubscribe **+0.31 pp**, Opt-out **+0.54 pp** (small increases; monitor against internal thresholds).

- **Segments:**  
  Strongest lift in **PS+ members with low spend (S1_low): +6.25 pp**.  
  PS+ members across tiers also show strong lift (~+4.9 to +5.6 pp).

- **Decision:**  
  **Ship** with guardrail monitoring.  
  **Next test:** Optimize messaging frequency/creative for non-PS+ users, and test incentive variants for PS+ low spend to maximize lift while controlling opt-outs.
